In [2]:
import itertools
import pandas as pd
import cobra.flux_analysis
import time
import numpy as np
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks

from functools import partial
from src.met_task_functions import constrain_model, read_tasks


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Human1 model as it takes some time to load in."""

def load_model(model_path: str) -> list:
    model = cobra.io.read_sbml_model(model_path)
    model_list = constrain_model(model, ALLMETSIN=True)
    #Removes unused model.
    model_list[2] = None
    return model_list


def get_unique_gene_combs(combs: pd.DataFrame) -> pd.DataFrame:

    df = pd.DataFrame(combs['gene_ids'], columns=['gene_ids'])
    f = list(np.unique(df))
    d = {}
    n=0
    for l in f:
        d[n] = [l]
        n += 1

    return pd.DataFrame.from_dict(d, orient='index', columns=['gene_ids'])


def produce_ind_combs(model_genes: list) -> pd.DataFrame:
    ind_data = pd.read_table('C:/Users/Sigve/Genome_Data/SNP_data/1000_genomes/result_chrom_22.tsv', index_col=0)
    missense_snps = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_missense.tsv', index_col=0)

    missense_snps['gene_id'] = missense_snps['gene_id'].apply(lambda x: x.split('.')[0])
    merged_df = pd.merge(left=missense_snps, right=ind_data, how='left', left_on='chrom_pos', right_on='pos')

    res_dict = {}

    for column in merged_df:
        res_dict[column] = [merged_df['variant_name'][merged_df[column] == 2].to_list(), set(merged_df['gene_id'][merged_df[column] == 2].to_list())]

    results = pd.DataFrame.from_dict(res_dict, orient='index', columns=['snp_list', 'gene_ids'])
    results['gene_ids'] = results['gene_ids'].apply(lambda x: list(x.intersection(model_genes)))
    results = results[results['gene_ids'].map(lambda x: len(x)) > 0]

    return results

In [ ]:
start_time = time.time()
# ['blood']
tissue_list = ['brain', 'heart', 'kidney', 'liver', 'muscle', 'nerve', 'pancreas', 'thyroid']
model_base_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/{0}.xml'

for tissue in tissue_list: 

    model_list = load_model(model_base_path.format(tissue))
    model_genes = [gene.id for gene in model_list[0].genes]

    task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)
    ind_combs = produce_ind_combs(model_genes)
    ind_combs['gene_ids'].apply(lambda x: x.sort())

    unique_combs = get_unique_gene_combs(ind_combs)

    results = parallelize_dataframe(unique_combs, partial(combinations_subset, partial(knockout_FBA_w_tasks, task_list, model_list)), 13)

    results['gene_ids'] = results['gene_ids'].apply(lambda x: ';'.join(x))
    results['solution'] = results['results'].apply(lambda x: x[0])
    results['tasks_results'] = results['results'].apply(lambda x: x[1:])

    ind_combs['gene_ids'] = ind_combs['gene_ids'].apply(lambda x: ';'.join(x))
    ind_combs.reset_index(drop=False, inplace=True)
    ind_combs['ind_id'] = ind_combs['index']

    results = pd.merge(left=ind_combs, right=results, how='right', left_on='gene_ids', right_on='gene_ids')
    results['gene_ids'] = results['gene_ids'].apply(lambda x: x.split(';'))




    results[['ind_id', 'solution', 'tasks_results']].to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/ind_results/ind_chrom_22_{0}.tsv'.format(tissue), sep='\t')


end_time = time.time()

print('Total time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpr_d2uv5b.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpomnktrt8.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp7a5m4470.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpvjj4pq9v.lp
Reading time = 0.07 seconds
: 5321 rows, 14488 columns, 63246 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpnkkgr6vc.lp
Reading time = 0.06 seconds
: 5321 rows, 14488 columns, 63246 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp3jsrusu4.lp
Reading time = 0.06 seconds
: 5321 rows, 14488 columns, 6

In [2]:
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


In [84]:
model_genes = [gene.id for gene in model.genes]
combs = produce_ind_combs(model_genes)

In [ ]:
print(combs.shape[0])

